In [1]:
import argparse
import sys
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np

FLAGS = None

In [2]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

In [3]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [4]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [5]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [6]:
a = np.array([[[1,1],[2,2],[3,3]],[[1,1],[2,2],[3,3]]])
print(a)
b = np.array([1,2])
print(np.shape(a))
print(np.shape(b))
c = a + b
print(c)

for epoch in range(10):
    print(epoch)

[[[1 1]
  [2 2]
  [3 3]]

 [[1 1]
  [2 2]
  [3 3]]]
(2, 3, 2)
(2,)
[[[2 3]
  [3 4]
  [4 5]]

 [[2 3]
  [3 4]
  [4 5]]]
0
1
2
3
4
5
6
7
8
9


In [7]:
def deepnn(x):
    with tf.name_scope('reshape'):
        x_image = tf.reshape(x, [-1, 28, 28, 1])
        
    with tf.name_scope('conv1'):
        W_conv1 = weight_variable([5, 5, 1, 32])
        b_conv1 = bias_variable([32])
        h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
        
    with tf.name_scope('pool1'):
        h_pool1 = max_pool_2x2(h_conv1)
        
    with tf.name_scope('conv2'):
        W_conv2 = weight_variable([5, 5, 32, 64])
        b_conv2 = bias_variable([64])
        h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2)+ b_conv2)
        
    with tf.name_scope('pool2'):
        h_pool2 = max_pool_2x2(h_conv2)
        
    with tf.name_scope('fc1'):
        h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 *64])
        W_fc1 = weight_variable([7 * 7 * 64, 1024])
        b_fc1 = bias_variable([1024])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
        
    with tf.name_scope('dropout'):
        keep_prob = tf.placeholder(tf.float32)
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
        
    with tf.name_scope('fc2'):
        W_fc2 = weight_variable([1024, 10])
        b_fc2 = bias_variable([10])
        
        y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    return y_conv, keep_prob
        

In [8]:
y1 = np.array([[1, 2, 3],[6, 5, 4]])
y2 = np.array([[0.3, 0.4, 0.9],[0.5, 0.6, 0.6]])
p = tf.equal(tf.argmax(y1, 1),tf.argmax(y2, 1))
p2 = tf.cast(p, tf.float32)
p3 = tf.reduce_mean(p2)

with tf.Session() as sess:
    print(sess.run(tf.argmax(y1, 1)))
    print(sess.run(tf.argmax(y2, 1)))
    print(sess.run(p))
    print(sess.run(p2))
    print(sess.run(p3))

[2 0]
[2 1]
[ True False]
[ 1.  0.]
0.5


In [23]:
def main(_):
    mnist = input_data.read_data_sets(FLAGS.data_dir, one_hot=True, validation_size=10000)
    
    x = tf.placeholder(tf.float32, [None, 784])
    
    y_ = tf.placeholder(tf.float32, [None, 10])
    
    y_conv, keep_prob = deepnn(x)
    
    with tf.name_scope('loss'):
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv)
        cross_entropy = tf.reduce_mean(cross_entropy)
        
    with tf.name_scope('adam_optimizer'):
        train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
        
    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
        correct_prediction = tf.cast(correct_prediction, tf.float32)
        accuracy = tf.reduce_mean(correct_prediction)
        
    graph_location = 'MNIST/logs/tf2-6/train'
    print('Saving graph to: %s' % graph_location)
    train_writer = tf.summary.FileWriter(graph_location)
    train_writer.add_graph(tf.get_default_graph())
    
    saver = tf.train.Saver()
    
    best = 0
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(60):
            for _ in range(1000):
                batch = mnist.train.next_batch(50)
                train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
            accuracy_validation = accuracy.eval(feed_dict={x: mnist.validation.images, y_: mnist.validation.labels, keep_prob: 1.0})
            print('epcho %d, validation accuracy %s' % (epoch+1, accuracy_validation))
            best = (best, accuracy_validation)[best <= accuracy_validation]
            
            saver.save(sess, 'MNIST/logs/tf2-6/checkpoint/model.ckpt')
            
    print("best: %s" % best)

In [ ]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_dir', type=str, default='../MNIST/',
                        help='Directory for storing input data')
    FLAGS, unparsed = parser.parse_known_args()
    tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

Extracting ../MNIST/train-images-idx3-ubyte.gz
Extracting ../MNIST/train-labels-idx1-ubyte.gz
Extracting ../MNIST/t10k-images-idx3-ubyte.gz
Extracting ../MNIST/t10k-labels-idx1-ubyte.gz
Saving graph to: MNIST/logs/tf2-6/train
epcho 1, validation accuracy 0.9616
